In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D # convolution layers
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers

from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import to_categorical

In [ ]:
train_path='/kaggle/input/recognizance-2/Data/train'

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,  
        zoom_range = 0.2,
        shear_range=0.2,
        rotation_range=20,
        horizontal_flip=True,  
        vertical_flip=False,
        validation_split=0.1
        ) 
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (128,128),
                                                 batch_size = 128,
                                                 color_mode='grayscale',
                                                 class_mode='binary',
                                                 subset='training'
                                                 )
valid_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (128,128),
                                                 batch_size = 128,
                                                 color_mode='grayscale',
                                                 class_mode='binary',
                                                 subset='validation'
                                                 )

In [ ]:
model= Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),input_shape= (128,128,1),activation='relu'))
model.add(MaxPool2D(pool_size =(2,2)))
#clf.add(BatchNormalization())


model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(MaxPool2D(pool_size =(2,2)))
#clf.add(BatchNormalization())


model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
model.add(MaxPool2D(pool_size =(2,2)))


model.add(Conv2D(256, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
#clf.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics =['accuracy'])

In [ ]:
history=model.fit(x=training_set, 
               validation_data = valid_set,
               batch_size=128,
               epochs = 100)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save("my_h5_model.h5")

In [ ]:
training_set.class_indices

In [ ]:
import os
from tensorflow.keras.preprocessing.image import img_to_array
X_aaa = np.zeros((6078,128,128,1))
w=0
directory = '/kaggle/input/recognizance-2/Data/test'
for filename in os.listdir(directory):
    print(filename)
    img=cv2.imread(directory+'/'+filename,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(128,128))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)
    X_aaa[w] = x
    w = w + 1

In [ ]:
X_aaa=X_aaa/255.0
print(X_aaa)

In [ ]:
x_pred_name = [] 
predicted_sub = []
pre = model.predict(X_aaa)
directory = '/kaggle/input/recognizance-2/Data/test'
for i,filename in enumerate(os.listdir(directory)):
    x_pred_name.append(filename)
    print(pre[i])
    if(pre[i]<0.5):
        predicted_sub.append('NO')
    else:
        predicted_sub.append('YES')
df_pred = pd.DataFrame(np.c_[x_pred_name,predicted_sub],columns=['image file name','Powerline'])
df_pred.to_csv('/kaggle/working/submisson.csv', index = False, header=True)